In [17]:
import requests
import json

In [19]:
access_token = !gcloud auth application-default print-access-token
access_token = '\n'.join(access_token)

project = 'boreal-ward-437408-p6'
base_url = "https://generativelanguage.googleapis.com"

In [20]:
headers={
  'Authorization': 'Bearer ' + access_token,
  'Content-Type': 'application/json',
  'x-goog-user-project': project
}

result = requests.get(
  url=f'{base_url}/v1beta/tunedModels',
  headers = headers,
)

In [21]:
import time

initial_instruction = (
    "You are a medical assistant"
    "Your goal is to answer questions related medical conditions."
    "For questions about health except for Atelectasis, Cardiomegaly, Effusion, Infiltration, Mass, Nodule, Pneumonia, Pneumothorax, "
    "Consolidation, Edema, Emphysema you should answer those questions, with the answer add the disclaimer 'disclaimer: I am only specilized in certian lung and chest diseases.' "
    "For other prompts not related to health, respond with: 'I can only assist with questions related to medical conditions, particularly those concerning lung and chest diseases. If your inquiry falls outside of health-related topics, I won’t be able to respond. Please keep the questions focused on medical conditions.'"
)

def ask_medical_chatbot(conversation_history):
    parts = [{"text": initial_instruction}]
    
    for exchange in conversation_history:
        parts.append({"text": exchange["user"]})
        if exchange["assistant"]:
            parts.append({"text": exchange["assistant"]})

    payload = {
        "contents": [{
            "parts": parts
        }]
    }

    response = requests.post(
        url='https://generativelanguage.googleapis.com/v1beta/tunedModels/book2modifiedanswers-6vt4z4xscfbz:generateContent',
        headers=headers,
        json=payload
    )
    
    if response.status_code == 200:
        response_json = response.json()
        try:
            answer = response_json['candidates'][0]['content']['parts'][-1]['text']
            return answer.strip() 
        except (KeyError, IndexError) as e:
            return "Error: Unable to parse the response structure."
    else:
        return f"Error: {response.status_code} - {response.text}"

def main():
    print("Welcome to the Medical Chatbot specialized in lung and chest conditions.")
    print("You can ask questions about Atelectasis, Cardiomegaly, and other related conditions.")
    print("Type 'exit' or 'quit' to end the session.\n")

    conversation_history = []

    while True:
        question = input("You: ").strip()

        if question.lower() in ['exit', 'quit']:
            print("Assistant: Thank you for using the Medical Chatbot. Goodbye!")
            break  

        if not question:
            print("Assistant: Please enter a valid question.\n")
            continue  

        conversation_history.append({"user": question, "assistant": ""})

        assistant_reply = ask_medical_chatbot(conversation_history)

        assistant_reply = assistant_reply.replace('*', '')

        conversation_history[-1]["assistant"] = assistant_reply

        print(f"Assistant: {assistant_reply}\n")

if __name__ == "__main__":
    main()


Welcome to the Medical Chatbot specialized in lung and chest conditions.
You can ask questions about Atelectasis, Cardiomegaly, and other related conditions.
Type 'exit' or 'quit' to end the session.

Assistant: An effusion refers to the abnormal accumulation of fluid in a body cavity, such as the pleural space surrounding the lungs. In the context of lung diseases, an effusion involves the buildup of fluid in the pleural space, leading to pleural effusion. This condition can be caused by various underlying factors, including infections, inflammatory diseases, heart failure, and malignancies. The presence of fluid in the pleural space can compress the lungs, hindering their ability to expand fully and compromising respiratory function.  Disclaimer: I am only specialized in certain lung and chest diseases.

Assistant: An effusion refers to the abnormal accumulation of fluid in a body cavity, such as the pleural space surrounding the lungs. In the context of lung diseases, an effusion in

In [22]:
# import requests
# import time

# # Instruction to be sent before every new chat session
# initial_instruction = (
#     "You are a medical assistant specialized in lung and chest diseases. "
#     "Your goal is to answer questions related to the symptoms, diagnosis, treatment, prevention, and complications "
#     "of the following conditions: Atelectasis, Cardiomegaly, Effusion, Infiltration, Mass, Nodule, Pneumonia, Pneumothorax, "
#     "Consolidation, Edema, Emphysema, Fibrosis, Pleural Thickening, Hernia. "
#     "For any other conditions or general health questions or any other prompts, respond with: 'I can only answer questions related to certain lung and chest conditions.' "
#     "If the question is about symptoms or involves symptom-based queries, provide a diagnosis or possible explanations within the scope of the listed diseases."
# )


# # Function to send the conversation history to the API
# def ask_medical_chatbot(conversation_history):
#     # Start building the conversation parts with the initial instruction at the start
#     parts = [{"text": initial_instruction}]
    
#     # Add all exchanges (user and assistant) from the conversation history
#     for exchange in conversation_history:
#         # User's question
#         parts.append({"text": exchange["user"]})
#         # Assistant's response, if available
#         if exchange["assistant"]:
#             parts.append({"text": exchange["assistant"]})

#     # Prepare the payload for the API call
#     payload = {
#         "contents": [{
#             "parts": parts
#         }]
#     }

#     # Make the API request
#     response = requests.post(
#         url='https://generativelanguage.googleapis.com/v1beta/tunedModels/book2modifiedanswers-6vt4z4xscfbz:generateContent',
#         headers=headers,
#         json=payload
#     )
    
#     # Handle the response from the API
#     if response.status_code == 200:
#         response_json = response.json()
#         try:
#             # Extract the assistant's reply
#             answer = response_json['candidates'][0]['content']['parts'][-1]['text']
#             return answer.strip()  # Remove any leading/trailing whitespace
#         except (KeyError, IndexError) as e:
#             return "Error: Unable to parse the response structure."
#     else:
#         return f"Error: {response.status_code} - {response.text}"

# def main():
#     print("Welcome to the Medical Chatbot specialized in lung and chest conditions.")
#     print("You can ask questions about Atelectasis, Cardiomegaly, and other related conditions.")
#     print("Type 'exit' or 'quit' to end the session.\n")

#     conversation_history = []
#     MAX_CONVERSATIONS = 20  # Maximum number of user-assistant exchanges

#     while True:
#         question = input("You: ").strip()

#         if question.lower() in ['exit', 'quit']:
#             print("Assistant: Thank you for using the Medical Chatbot. Goodbye!")
#             break  # Exit the loop and terminate the program

#         if not question:
#             print("Assistant: Please enter a valid question.\n")
#             time.sleep(2)  # Wait for 2 seconds before prompting the user again
#             continue  # Prompt the user again if input is empty

#         # Check if the maximum number of conversations has been reached
#         if len(conversation_history) >= MAX_CONVERSATIONS:
#             print("Assistant: We have reached the maximum number of interactions (20) for this session.")
#             print("Assistant: Thank you for using the Medical Chatbot. Goodbye!")
#             break  # Exit the loop and terminate the program

#         # Add the user's question to the conversation history
#         conversation_history.append({"user": question, "assistant": ""})

#         # Get the assistant's reply
#         assistant_reply = ask_medical_chatbot(conversation_history)

#         # Remove asterisks from the assistant's reply
#         assistant_reply = assistant_reply.replace('*', '')

#         # Update the last entry in conversation history with assistant's reply
#         conversation_history[-1]["assistant"] = assistant_reply

#         print(f"Assistant: {assistant_reply}\n")

#     # Optionally, you can save the conversation history or perform other cleanup tasks here
#     # For example:
#     # save_conversation(conversation_history)

# if __name__ == "__main__":
#         main()


Welcome to the Medical Chatbot specialized in lung and chest conditions.
You can ask questions about Atelectasis, Cardiomegaly, and other related conditions.
Type 'exit' or 'quit' to end the session.

Assistant: Thank you for using the Medical Chatbot. Goodbye!


In [23]:
# import requests
# import time

# # Instruction to be sent before every new chat session
# initial_instruction = (
#     "You are a medical assistant specialized in lung and chest diseases. "
#     "Your goal is to answer questions related to the symptoms, diagnosis, treatment, prevention, and complications "
#     "of the following conditions: Atelectasis, Cardiomegaly, Effusion, Infiltration, Mass, Nodule, Pneumonia, Pneumothorax, "
#     "Consolidation, Edema, Emphysema, Fibrosis, Pleural Thickening, Hernia. "
#     "For any other conditions or general health questions or any other prompts, respond with: 'I can only answer questions related to certain lung and chest conditions.' "
#     "If the question is about symptoms or involves symptom-based queries, provide a diagnosis or possible explanations within the scope of the listed diseases."
# )

# # Function to send the conversation history to the API
# def ask_medical_chatbot(conversation_history):
#     # Start building the conversation parts with the initial instruction at the start
#     parts = [{"text": initial_instruction}]
    
#     # Add all exchanges (user and assistant) from the conversation history
#     for exchange in conversation_history:
#         # User's question
#         parts.append({"text": exchange["user"]})
#         # Assistant's response, if available
#         if exchange["assistant"]:
#             parts.append({"text": exchange["assistant"]})

#     # Prepare the payload for the API call
#     payload = {
#         "contents": [{
#             "parts": parts
#         }]
#     }

#     # Make the API request
#     response = requests.post(
#         url='https://generativelanguage.googleapis.com/v1beta/tunedModels/book2modifiedanswers-6vt4z4xscfbz:generateContent',
#         headers=headers,
#         json=payload
#     )
    
#     # Handle the response from the API
#     if response.status_code == 200:
#         response_json = response.json()
#         try:
#             # Extract the assistant's reply
#             answer = response_json['candidates'][0]['content']['parts'][-1]['text']
#             return answer.strip()  # Remove any leading/trailing whitespace
#         except (KeyError, IndexError) as e:
#             return "Error: Unable to parse the response structure."
#     else:
#         return f"Error: {response.status_code} - {response.text}"

# def main():
#     print("Welcome to the Medical Chatbot specialized in lung and chest conditions.")
#     print("You can ask questions about Atelectasis, Cardiomegaly, and other related conditions.")
#     print("Type 'exit' or 'quit' to end the session.\n")

#     conversation_history = []

#     while True:
#         question = input("You: ").strip()

#         if question.lower() in ['exit', 'quit']:
#             print("Assistant: Thank you for using the Medical Chatbot. Goodbye!")
#             break  # Exit the loop and terminate the program

#         # if not question:
#         #     print("Assistant: Please enter a valid question.")
#         #     time.sleep(2)  # Wait for 2 seconds before prompting the user again
#         #     continue  # Prompt the user again if input is empty

#         # Add the user's question to the conversation history
#         conversation_history.append({"user": question, "assistant": ""})

#         # Get the assistant's reply
#         assistant_reply = ask_medical_chatbot(conversation_history)

#         # Update the last entry in conversation history with assistant's reply
#         conversation_history[-1]["assistant"] = assistant_reply

#         print(f"Assistant: {assistant_reply}\n")

# if __name__ == "__main__":
#     main()
